# LETS DO IT BOYSSSSSS

In [77]:
import gzip
import json
import random
import numpy as np
import pandas as pd

In [57]:
def reading_gz(filename):
    f = gzip.open(filename, 'rt', encoding="utf8")
    lines = []
    for line in f:
        liness = json.loads(line.strip())
        lines.append(liness)
    return lines

In [96]:
data_modcloth = reading_gz('modcloth_final_data.json.gz')
data_renttherunway = reading_gz('renttherunway_final_data.json.gz')

The data_modcloth has 82790 data and the data_renttherunway has 192544 items

In [66]:
len(data_modcloth),len(data_renttherunway)

(82790, 192544)

In [74]:
data_modcloth[:3]

[{'item_id': '123373',
  'waist': '29',
  'size': 7,
  'quality': 5,
  'cup size': 'd',
  'hips': '38',
  'bra size': '34',
  'category': 'new',
  'bust': '36',
  'height': '5ft 6in',
  'user_name': 'Emily',
  'length': 'just right',
  'fit': 'small',
  'user_id': '991571'},
 {'item_id': '123373',
  'waist': '31',
  'size': 13,
  'quality': 3,
  'cup size': 'b',
  'hips': '30',
  'bra size': '36',
  'category': 'new',
  'length': 'just right',
  'height': '5ft 2in',
  'user_name': 'sydneybraden2001',
  'fit': 'small',
  'user_id': '587883'},
 {'item_id': '123373',
  'waist': '30',
  'size': 7,
  'quality': 2,
  'cup size': 'b',
  'shoe size': '9.00',
  'bra size': '32',
  'category': 'new',
  'length': 'slightly long',
  'height': '5ft 7in',
  'user_name': 'Ugggh',
  'fit': 'small',
  'user_id': '395665'}]

The data in data_modcloth has the user_name, userID, user_size, fit

In [69]:
data_renttherunway[0]

{'fit': 'fit',
 'user_id': '420272',
 'bust size': '34d',
 'item_id': '2260466',
 'weight': '137lbs',
 'rating': '10',
 'rented for': 'vacation',
 'review_text': "An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.",
 'body type': 'hourglass',
 'review_summary': 'So many compliments!',
 'category': 'romper',
 'height': '5\' 8"',
 'size': 14,
 'age': '28',
 'review_date': 'April 20, 2016'}

The data contain review_text, user_id, item_id, weight, and rating

#  data_renttherunway

In [108]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy
import urllib
import scipy.optimize
import random
from collections import defaultdict # Dictionaries with default values
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model
import ast
import numpy as np

In [98]:
np.random.shuffle(data_renttherunway)

In [99]:
num_90th = int(len(data_renttherunway)*0.9)
num_90th

173289

In [100]:
train_set = data_renttherunway[0:num_90th]
validation_set = data_renttherunway[num_90th:]

change the category to numerical data

In [110]:
categorys= np.unique([w['category'] for w in data_renttherunway])
categoryId = dict(zip(categorys, 
                  range(len(categorys))))

In [118]:
categorys

array(['ballgown', 'blazer', 'blouse', 'blouson', 'bomber', 'buttondown',
       'caftan', 'cami', 'cape', 'cardigan', 'coat', 'combo', 'crewneck',
       'culotte', 'culottes', 'down', 'dress', 'duster', 'for', 'frock',
       'gown', 'henley', 'hoodie', 'jacket', 'jeans', 'jogger',
       'jumpsuit', 'kaftan', 'kimono', 'knit', 'legging', 'leggings',
       'maxi', 'midi', 'mini', 'overalls', 'overcoat', 'pant', 'pants',
       'parka', 'peacoat', 'poncho', 'print', 'pullover', 'romper',
       'sheath', 'shift', 'shirt', 'shirtdress', 'skirt', 'skirts',
       'skort', 'suit', 'sweater', 'sweatershirt', 'sweatpants',
       'sweatshirt', 't-shirt', 'tank', 'tee', 'tight', 'top', 'trench',
       'trouser', 'trousers', 'tunic', 'turtleneck', 'vest'], dtype='<U12')

There are some repeated meaning category, so we need to stem the category

In [113]:
stemmer = PorterStemmer()

In [114]:
category_stem= np.unique([stemmer.stem(w['category'])for w in data_renttherunway])
categorystemId = dict(zip(categorys, 
                  range(len(categorys))))

In [119]:
category_stem

array(['ballgown', 'blazer', 'blous', 'blouson', 'bomber', 'buttondown',
       'caftan', 'cami', 'cape', 'cardigan', 'coat', 'combo', 'crewneck',
       'culott', 'down', 'dress', 'duster', 'for', 'frock', 'gown',
       'henley', 'hoodi', 'jacket', 'jean', 'jogger', 'jumpsuit',
       'kaftan', 'kimono', 'knit', 'leg', 'maxi', 'midi', 'mini',
       'overal', 'overcoat', 'pant', 'parka', 'peacoat', 'poncho',
       'print', 'pullov', 'romper', 'sheath', 'shift', 'shirt',
       'shirtdress', 'skirt', 'skort', 'suit', 'sweater', 'sweatershirt',
       'sweatpant', 'sweatshirt', 't-shirt', 'tank', 'tee', 'tight',
       'top', 'trench', 'trouser', 'tunic', 'turtleneck', 'vest'],
      dtype='<U12')

Moreover, there is still has some repeated categories such as t_shirt and tee, blous and blouson, sweater, sweatershirt and sweatshirt. Therefore, we need to clear the category type futher

In [120]:
x_text_train = [d['review_text'] for d in train_set]
y_text_train = [d['category'] for d in train_set]
x_text_test = [d['review_text'] for d in validation_set]
y_text_test = [d['category'] for d in validation_set]